# Geospatial Python
## Accessing satellite imagery using Python
Setup: https://carpentries-incubator.github.io/geospatial-python/index.html

Instruction: https://carpentries-incubator.github.io/geospatial-python/05-access-data.html

Objectives:
* Search public [SpatioTemporal Asset Catalog (STAC)](https://github.com/radiantearth/stac-api-spec/tree/release/v1.0.0) repositories of satellite imagery using Python.
* Inspect search result’s metadata.
* Download (a subset of) the assets available for a satellite scene.
* Open satellite imagery as raster data and save it to disk.

In [1]:
# first import necessary libraries
import rioxarray # to open and download remote raster data
from pystac_client import Client # to query STAC API endpoint
from shapely.geometry import Point # to create a point 

In [2]:
# get the source url (top right button) from https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1
# to access the STAC catalog items
api_url = "https://earth-search.aws.element84.com/v1"

# open the api
client = Client.open(api_url)
# see documentation https://pystac-client.readthedocs.io/en/stable/


In [3]:
# perform metadata search limited to 10 results from Sentinel-2, Level 2A, to retrieve Cloud Optimized GeoTiffs (COGs)

# store a variable pointing to the collection of interest
# Note: collection ID is taken from Sentinel-2 Level 2A - https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1/collections/sentinel-2-c1-l2a
collection = "sentinel-2-l2a" 
'''
includes Sentinel-2 data products 
pre-processed at level 2A (bottom-of-atmosphere reflectance) 
and saved in Cloud Optimized GeoTIFF (COG) format:
'''

# create a point to intersect from, note values are in format x (long), y (lat) https://shapely.readthedocs.io/en/stable/reference/shapely.Point.html
point = Point(4.89, 52.37)  # AMS (Amsterdam Airport Schiphol) coordinates, use https://www.google.com/maps

search = client.search(
    #collections=[collection],
    intersects=point,
    max_items=10,
)

In [4]:
# show the number of scenes (i.e. the portion of the footage recorded by the satellite)
print(search.matched())

7300


In [5]:
# store the metadata of the search results
items = search.item_collection()

In [6]:
# get the length of items
print(len(items))

10


In [7]:
# loop over the items to get there ids
for item in items:
    print(item)

<Item id=S2A_T31UFU_20240206T104338_L2A>
<Item id=S2A_31UFU_20240206_0_L2A>
<Item id=S2A_31UFU_20240206_0_L1C>
<Item id=S1A_IW_GRDH_1SDV_20240206T055038_20240206T055103_052434_06574F>
<Item id=S2B_T31UFU_20240204T105432_L2A>
<Item id=S2B_31UFU_20240204_0_L2A>
<Item id=S2B_31UFU_20240204_0_L1C>
<Item id=S1A_IW_GRDH_1SDV_20240202T173335_20240202T173400_052383_06559F>
<Item id=S2B_T31UFU_20240201T104520_L2A>
<Item id=S2B_31UFU_20240201_0_L2A>


In [8]:
#  inspect the metadata associated with the first item of the search result
item = items[0]
print(item.datetime)
print(item.geometry)
print(item.properties)

2024-02-06 10:46:21.552000+00:00
{'type': 'Polygon', 'coordinates': [[[5.252006216419567, 53.22834088045702], [4.827001791442609, 52.24841727934954], [6.071664488869862, 52.22257539160585], [6.141754296879459, 53.20819279121764], [5.252006216419567, 53.22834088045702]]]}
{'created': '2024-02-06T17:16:42.755Z', 'platform': 'sentinel-2a', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'eo:cloud_cover': 99.976689, 'proj:epsg': 32631, 'proj:centroid': {'lat': 52.69969, 'lon': 5.56608}, 'mgrs:utm_zone': 31, 'mgrs:latitude_band': 'U', 'mgrs:grid_square': 'FU', 'grid:code': 'MGRS-31UFU', 'view:azimuth': 106.87922568746447, 'view:incidence_angle': 9.021636724016243, 'view:sun_azimuth': 162.959552738812, 'view:sun_elevation': 20.0724861346243, 's2:tile_id': 'S2A_OPER_MSI_L2A_TL_2APS_20240206T145748_A045050_T31UFU_N05.10', 's2:degraded_msi_data_percentage': 0.017, 's2:nodata_pixel_percentage': 34.102488, 's2:saturated_defective_pixel_percentage': 0, 's2:dark_features_percentage': 0, 's2:

In [9]:
'''
EXERCISE: Search the sentinel-2-l2a collection for all the available scenes that satisfy the following criteria: 
- intersect a provided bounding box (use ±0.01 deg in lat/lon from the previously defined point); 
- have been recorded between 20 March 2020 and 30 March 2020; 
- have a cloud coverage less than 15. Note: the eo extension (https://github.com/stac-extensions/eo) is implemented in some collections allowing it to be queried against

* get the count
* save the results to json
'''
bbox = point.buffer(0.01).bounds

search = client.search(
    collections=[collection],
    bbox=bbox,
    datetime="2020-03-20/2020-03-30",
    query=["eo:cloud_cover<15"]
)
print(search.matched())
items = search.item_collection()
items.save_object("search.json") # json file saved alongside notebook

6


## Access the assets


In [10]:
# first item's assets
assets = items[0].assets  

print(assets.keys())

dict_keys(['aot', 'blue', 'coastal', 'granule_metadata', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1', 'rededge2', 'rededge3', 'scl', 'swir16', 'swir22', 'thumbnail', 'tileinfo_metadata', 'visual', 'wvp', 'aot-jp2', 'blue-jp2', 'coastal-jp2', 'green-jp2', 'nir-jp2', 'nir08-jp2', 'nir09-jp2', 'red-jp2', 'rededge1-jp2', 'rededge2-jp2', 'rededge3-jp2', 'scl-jp2', 'swir16-jp2', 'swir22-jp2', 'visual-jp2', 'wvp-jp2'])


In [11]:
# print a minimal description of the available assets
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

aot: Aerosol optical thickness (AOT)
blue: Blue (band 2) - 10m
coastal: Coastal aerosol (band 1) - 60m
granule_metadata: None
green: Green (band 3) - 10m
nir: NIR 1 (band 8) - 10m
nir08: NIR 2 (band 8A) - 20m
nir09: NIR 3 (band 9) - 60m
red: Red (band 4) - 10m
rededge1: Red edge 1 (band 5) - 20m
rededge2: Red edge 2 (band 6) - 20m
rededge3: Red edge 3 (band 7) - 20m
scl: Scene classification map (SCL)
swir16: SWIR 1 (band 11) - 20m
swir22: SWIR 2 (band 12) - 20m
thumbnail: Thumbnail image
tileinfo_metadata: None
visual: True color image
wvp: Water vapour (WVP)
aot-jp2: Aerosol optical thickness (AOT)
blue-jp2: Blue (band 2) - 10m
coastal-jp2: Coastal aerosol (band 1) - 60m
green-jp2: Green (band 3) - 10m
nir-jp2: NIR 1 (band 8) - 10m
nir08-jp2: NIR 2 (band 8A) - 20m
nir09-jp2: NIR 3 (band 9) - 60m
red-jp2: Red (band 4) - 10m
rededge1-jp2: Red edge 1 (band 5) - 20m
rededge2-jp2: Red edge 2 (band 6) - 20m
rededge3-jp2: Red edge 3 (band 7) - 20m
scl-jp2: Scene classification map (SCL)
swi

In [12]:
# show one metadata value
print(assets["thumbnail"])
print(assets["thumbnail"].href)

<Asset href=https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/31/U/FU/2020/3/S2A_31UFU_20200328_1_L2A/thumbnail.jpg>
https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/31/U/FU/2020/3/S2A_31UFU_20200328_1_L2A/thumbnail.jpg


In [13]:
# open nir with the rioxarray library
nir_href = assets["nir"].href
nir = rioxarray.open_rasterio(nir_href)
print(nir)

<xarray.DataArray (band: 1, y: 10980, x: 10980)>
[120560400 values with dtype=uint16]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05 7.098e+05
  * y            (y) float64 5.9e+06 5.9e+06 5.9e+06 ... 5.79e+06 5.79e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  AVERAGE
    _FillValue:          0
    scale_factor:        1.0
    add_offset:          0.0


In [14]:
# save whole image to disk - this may take awhile
nir.rio.to_raster("nir.tif")

In [15]:
# save portion of an image to disk
nir[0,1500:2200,1500:2200].rio.to_raster("nir_subset.tif")

In [16]:
'''
Exercise: 
Using pystac_client, connect to the STAC endpoint https://lpdaac.usgs.gov/products/hlsl30v002/ 
- search for all assets of the Landsat 8 collection (HLSL30.v2.0) 
- from February to March 2021, 
- intersecting the point with longitude/latitude coordinates (-73.97, 40.78) deg.
* Visualize an item’s thumbnail (asset key browse).

'''
cmr_api_url = "https://cmr.earthdata.nasa.gov/stac/LPCLOUD"
client = Client.open(cmr_api_url)

# setup search
search = client.search(
    collections=["HLSL30.v2.0"],
    intersects=Point(-73.97, 40.78),
    datetime="2021-02-01/2021-03-30",
) # nasa cmr cloud cover filtering is currently broken: https://github.com/nasa/cmr-stac/issues/239

# retrieve search results
items = search.item_collection()
print(len(items))

5


In [17]:
# sort by cloud cover and show details for the least cloudy image
items_sorted = sorted(items, key=lambda x: x.properties["eo:cloud_cover"])
item = items_sorted[0]
print(item)

<Item id=HLS.L30.T18TWL.2021039T153324.v2.0>


In [18]:
# show the image url
print(item.assets["browse"].href)

https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-public/HLSL30.020/HLS.L30.T18TWL.2021039T153324.v2.0/HLS.L30.T18TWL.2021039T153324.v2.0.jpg


### Final note: Public metadata does not mean public data.
Consider getting a free NASA Earthdata account here https://urs.earthdata.nasa.gov/
And creating a netrc file for access here https://git.earthdata.nasa.gov/projects/LPDUR/repos/daac_data_download_python/browse/EarthdataLoginSetup.py

Then using 
*import os<br>
os.environ["GDAL_HTTP_COOKIEFILE"] = "./cookies.txt"<br>
os.environ["GDAL_HTTP_COOKIEJAR"] = "./cookies.txt"*
